# Kensho Kuremoto
## Part2:Weather API alternative
### Any comments without numbers are just there to explain what’s going on in my head.

## Q1)Examine both Open Meteo and Tomorrow and decide which one you'd prefer to use. What drove your decision?
# I chose Open Meteo :https://open-meteo.com/
### Because,
##### I thought Open Meteo might be easier to use because it lets you specify only the elements you need. Tomorrow doesn’t seem bad either, but narrowing down the elements makes it easier to understand the structure of the JSON data and likely reduces bugs as well.Also, since the API key is only needed for commercial use, it’s a big plus that we don’t need one for this homework.

# ★The API key is not used here, so I haven't used .env processing.

## Q2)URL to the documentation: https://open-meteo.com/en/docs

## Q3)Make a request for the current weather where you are born, or somewhere you've lived.
#### I entered the exact latitude and longitude of my real hometown, Osaka, Japan.

In [42]:
import requests

url = "https://api.open-meteo.com/v1/forecast"
hometown = {
    "latitude": 34.6938,
    "longitude": 135.5011,
    "current": "temperature_2m,apparent_temperature",
}

response = requests.get(url, params=hometown)

if response:
    data = response.json()
    temp_c = data.get("current", {}).get("temperature_2m")
    if temp_c:
        print(f"My hometown, Osaka is {temp_c}°C")


My hometown, Osaka is 19.7°C


In [50]:
#Amazingly, I realized that the API data only contains weather information in the form of weather codes, which need to be converted into actual descriptions. 
#At this point, I swore never to use Open Meteo again.

import requests

lat = 34.6938
lon = 135.5011

#There was a list of weather codes at the very bottom of the documentation.
url = "https://api.open-meteo.com/v1/forecast"
hometown = {
    "latitude": lat,
    "longitude": lon,
    "current": "temperature_2m,apparent_temperature,weather_code",
}

#Create a dictionary that defines the descriptions corresponding to each weather_code (Rely on ChatGPT's help for this part).
weather_code_map = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing rime fog",
    51: "Light drizzle",
    53: "Moderate drizzle",
    55: "Dense drizzle",
    56: "Light freezing drizzle",
    57: "Dense freezing drizzle",
    61: "Slight rain",
    63: "Moderate rain",
    65: "Heavy rain",
    66: "Light freezing rain",
    67: "Heavy freezing rain",
    71: "Slight snow fall",
    73: "Moderate snow fall",
    75: "Heavy snow fall",
    77: "Snow grains",
    80: "Slight rain showers",
    81: "Moderate rain showers",
    82: "Violent rain showers",
    85: "Slight snow showers",
    86: "Heavy snow showers",
    95: "Thunderstorm",
    96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail"
}

response = requests.get(url, params=hometown)

if response.status_code:
    data = response.json()
    weather_code = data.get("current", {}).get("weather_code")
    
    if weather_code is not None:
        description = weather_code_map.get(weather_code, "Unknown weather code")
        print(f"The current weather in Osaka: {description}")


The current weather in Osaka: Moderate drizzle


## Q4)Print out the country this location is in.

In [20]:
import requests

#Unfortunately, the API data didn’t include a country name field. So, I need to identify the country using the latitude and longitude.

latitude = 34.6938
longitude = 135.5011

#Using Nominatim to identify the country this location is in.
url = "https://nominatim.openstreetmap.org/search"


country = {
    "q": f"{latitude}, {longitude}",
    "format": "jsonv2",
    "addressdetails": 1,
    "limit": 1
}

#The country name is output in Japanese, so I will switch it to English.
headers = {
    "User-Agent": "MyApp/1.0 (kensho19940928@gmail.com)",
    "Accept-Language": "en" 
}

response = requests.get(url, params=country, headers=headers)

if response.ok:
    data = response.json()
    if data and "address" in data[0]:
        country = data[0]["address"].get("country", "Unknown")
        print(f"My hometown, Osaka is in {country}.")
    else:
        print("Address information not found.")
else:
    print(f"Request failed: {response.status_code}")


My hometown, Osaka is in Japan.


## Q5)Print out the difference between the current temperature and how warm it feels. 

In [18]:
import requests

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 34.6938,
    "longitude": 135.5011,
    "current": "temperature_2m,apparent_temperature",
}

response = requests.get(url, params=params)
data = response.json()

temp = data["current"]["temperature_2m"]
apparent = data["current"]["apparent_temperature"]

diff = apparent - temp

#The decimal part is ridiculously long, so I’ll round it to one decimal place.
if diff > 0:
    print(f"It feels {diff:.1f} degrees warmer.")
elif diff < 0:
    print(f"It feels {abs(diff):.1f} degrees colder.")


It feels 2.8 degrees warmer.


## Q6)What's the current temperature at Heathrow International Airport? 

In [23]:
import requests

#Enter the Heathrow Airport IATA code “LHR” into Open Meteo, the following latitude and longitude are shown.
latitude = 51.4712
longitude = -0.4565

url = "https://api.open-meteo.com/v1/forecast"
weathernow = {
    "latitude": latitude,
    "longitude": longitude,
    "current_weather": True, 
}

response = requests.get(url, params=weathernow)

if response:
    data = response.json()
    current_temp = data.get("current_weather", {}).get("temperature")
    if current_temp:
        print(f"Heathrow International Airport current temperature: {current_temp}°C")


Heathrow International Airport current temperature: 15.4°C


## Q7)URL to request a 3-day forecast at Heathrow 
#### :https://api.open-meteo.com/v1/forecast?latitude=51.4712&longitude=-0.4565&daily=temperature_2m_max,weather_code&current=temperature_2m&forecast_days=3

# Q8)Print the date of each of the 3 days you're getting a forecast for.

In [26]:
import requests

lat = 51.4712
lon = -0.4565

url = "https://api.open-meteo.com/v1/forecast"
forecast = {
    "latitude": lat,
    "longitude": lon,
    "daily": "temperature_2m_max,weather_code",
    "forecast_days": 3,
}

response = requests.get(url, params=forecast)
data = response.json()

dates = data.get("daily", {}).get("time", [])

print("3-day forecast dates:")
for date in dates:
    print(date)


3-day forecast dates:
2025-06-09
2025-06-10
2025-06-11


## Q9)Print the maximum temperature of each of the days.

In [30]:
import requests

lat = 51.4712
lon = -0.4565

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": lat,
    "longitude": lon,
    "daily": "temperature_2m_max,weather_code",
    "forecast_days": 3,
}

response = requests.get(url, params=params)

if response:
    data = response.json()
    forecast_days = data["daily"]["time"] 
    max_temps = data["daily"]["temperature_2m_max"] 

    for i in range(len(forecast_days)):
        date = forecast_days[i]
        max_temp = max_temps[i]
        print(f"Date: {date}")
        print(f"Max Temp: {max_temp}°C")


Date: 2025-06-09
Max Temp: 19.4°C
Date: 2025-06-10
Max Temp: 21.7°C
Date: 2025-06-11
Max Temp: 24.3°C


# Q10)Print only the day with the highest maximum temperature.

In [34]:

if response:
    data = response.json()
    dates = data["daily"]["time"]
    max_temps = data["daily"]["temperature_2m_max"]
  
    max_temp_value = max(max_temps)
    max_index = max_temps.index(max_temp_value)
    max_temp_date = dates[max_index]

    print(f"Date: {max_temp_date}")
    print(f"Max Temp: {max_temp_value}°C")


Date: 2025-06-11
Max Temp: 24.3°C


## Q11)Did you find this easier or more difficult than using the weatherapi.com, and why? Which would you recommend to someone interesting in building a tool around weather information?

### I genuinely believe that weatherapi.com is far easier to use and more recommendable than Open Meteo. In fact, I found Open Meteo so unfriendly that I would prefer not to use it again. My reasons are mainly the following two:

#### 1.The Required Elements Were Missing, and It Took an Incredible Amount of Effort
    The most difficult part of this assignment was extracting the weather condition and country name. The weather condition was only provided as a numeric weather code—an abstract number that required me to manually copy a list from the documentation and create a new dictionary just to make it human-readable. I clearly remember that the WeatherAPI immediately provided this information in plain language, so this felt like the most pointless task I’ve done since arriving in New York. It makes no sense to hide such a critical piece of information—the current readable weather condition—behind a code.
    
    Additionally, the country name wasn’t included in the API data at all. I had to take an entirely different approach just to determine the country from latitude and longitude. Comparing this to WeatherAPI, which did include this information, the process felt like a total waste of time.
#### 2.Lack of User-Friendliness and Documentation
    Open Meteo’s API documentation is not intuitive. It was hard to find the exact parameters and endpoints I needed. Compared to weatherapi.com, which offers clear examples and structured guides, Open Meteo felt like a guessing game. Even small tasks like getting current weather data involved unnecessary trial and error.
